# Deep Convolutional VAE

In [ ]:
import os
import sys
import datetime as dt

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('datasets/MNIST/', one_hot=True)

### Hyperparameters

In [ ]:
# Inputs
image_size = 28
image_channel = 1
image_size_flat = image_size * image_size * image_channel

# Network
filter_size = 5
conv1_size = 32
conv2_size = 64
fc1_size = 1024
hidden_dim = 256
latent_dim = 128
keep_prob = 0.8

# Training
learning_rate = 1e-3
batch_size = 24
iterations = 10000
log_interval = 1000
save_interval = 100

### Helper functions

In [ ]:
# weights
def weight(shape, name):
    initial = tf.truncated_normal(shape=shape, mean=0, stddev=0.4)
    return tf.Variable(initial, name=name)

# biases
def bias(shape, name):
    initial = tf.zeros(shape=[shape])
    return tf.Variable(initial, name=name)

# convolutional block
def conv(X, W, b):
    layer = tf.nn.conv2d(X, W, strides=[1, 1, 1, 1], padding='SAME')
    layer = layer + b  # add bias
    layer = tf.nn.relu(layer)
    layer = tf.nn.max_pool(layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    return layer

# deconvolutional block
def deconv(X, W, b):
    pass

# fully connected block
def dense(X, W, b, activation=tf.nn.relu):
    layer = tf.matmul(X, W) + b
    if activation:
        layer = activation(layer)
    return layer

# flatten
def flatten(layer):
    shape = layer.get_shape()
    features = np.array(shape[1:4], dtype=int).prod()
    layer = tf.reshape(layer, [-1, features])
    return layer, features

# Plot images in grid
def plot_images(imgs, size=28, name=None, **kwargs):
    grid = int(np.sqrt(len(imgs)))
    fig, axes = plt.subplots(grid, grid)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)
    if name:
        plt.suptitle(name)
    for i, ax in enumerate(axes.flat):
        ax.imshow(imgs[i].reshape([size, size]), **kwargs)
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

In [ ]:
def encoder(image):
    with tf.name_scope('encoder'):
        image = tf.reshape(image, [-1, image_size, image_size, image_channel])
        # 1st conv layer
        W_conv1 = weight(shape=[filter_size, filter_size, image_channel, conv1_size], name='W_conv1')
        b_conv1 = bias(shape=conv1_size, name='b_conv1')
        conv1 = conv(image, W_conv1, b_conv1)
        # 2nd conv layer
        W_conv2 = weight(shape=[filter_size, filter_size, conv1_size, conv2_size], name='W_conv2')
        b_conv2 = bias(shape=conv2_size, name='b_conv2')
        conv2 = conv(conv1, W_conv2, b_conv2)
        # Flatten
        flattened, feature = flatten(conv2)
        # Fully connected 1
        W_fc1 = weight(shape=[feature, fc1_size], name='W_fc1')
        b_fc1 = bias(shape=fc1_size, name='b_fc1')
        fc1 = dense(flattened, W_fc1, b_fc1)
        # Fully connected 2
        W_fc2 = weight(shape=[fc1_size, hidden_dim], name='W_fc2')
        b_fc2 = bias(shape=hidden_dim, name='b_fc2')
        fc2 = dense(fc1, W_fc2, b_fc2)
        # Mean
        W_mean = weight(shape=[hidden_dim, latent_dim], name='W_mean')
        b_mean = bias(shape=latent_dim, name='b_mean')
        mean = dense(fc2, W_mean, b_mean, activation=None)
        # Stddev
        W_stddev = weight(shape=[hidden_dim, latent_dim], name='W_stddev')
        b_stddev = bias(shape=latent_dim, name='b_stddev')
        stddev = 0.5 * dense(fc2, W_stddev, b_stddev, activation=None)
        # Random noise
        noise = tf.random_normal(shape=[1, latent_dim])
        encoded = mean + tf.multiply(noise, tf.exp(0.5 * stddev))
        return encoded, mean, stddev

In [ ]:
def decoder(encoded):
    with tf.name_scope('decoder'):
        # Fully connected 1
        W_fc1 = weight(shape=[latent_dim, hidden_dim], name='W_fc1')
        b_fc1 = bias(shape=hidden_dim, name='b_fc1')
        fc1 = dense(encoded, W_fc1, b_fc1, activation=tf.nn.tanh)
        # Fully connected 2
        W_fc2 = weight(shape=[hidden_dim, fc1_size], name='W_fc2')
        b_fc2 = bias(shape=fc1_size, name='b_fc2')
        fc2 = dense(fc1, W_fc2, b_fc2, activation=tf.nn.tanh)
        # Reconstruction
        W_rec = weight(shape=[fc1_size, image_size_flat], name='W_rec')
        b_rec = bias(shape=image_size_flat, name='b_rec')
        decoded = dense(fc2, W_rec, b_rec, activation=tf.nn.sigmoid)
        return decoded

In [ ]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, image_size_flat])

encoded, mean, stddev = encoder(X)
decoded = decoder(encoded)

### Reconstruction loss, KL-Divergence & optimizer

In [ ]:
rec_loss = tf.reduce_sum(tf.squared_difference(decoded, X), reduction_indices=1)
kl_term = -0.5 * tf.reduce_sum(1 + 2 * stddev - tf.square(mean) - tf.exp(2 * stddev), reduction_indices=1)
loss = tf.reduce_mean(rec_loss + kl_term)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(loss)

## Tensorflow's `tf.Session()`

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

### Tensorboard

In [ ]:
tensorboard_dir = 'tensorboard/'
logdir = os.path.join(tensorboard_dir, 'log')
save_path = 'models/'

tf.summary.histogram('rec_loss', rec_loss)
tf.summary.histogram('kl_term', kl_term)

tf.summary.scalar('loss', loss)
tf.summary.scalar('kl_term_mean', tf.reduce_mean(kl_term))
tf.summary.scalar('rec_loss_mean', tf.reduce_mean(rec_loss))
rec_img = tf.reshape(decoded, [-1, image_size, image_size, image_channel])
tf.summary.image('rec_img', rec_img, max_outputs=4)
merged = tf.summary.merge_all()

saver = tf.train.Saver()
writer = tf.summary.FileWriter(logdir=logdir, graph=sess.graph)

if tf.gfile.Exists(save_path):
    if len(tf.gfile.ListDirectory(save_path)) > 1:
        saver.restore(sess, save_path)
else:
    tf.gfile.MakeDirs(save_path)

## Training

In [ ]:
start_time = dt.datetime.now()
for i in range(iterations):
    batch = data.train.next_batch(batch_size=batch_size)[0]
    _, _kl_loss, _rec_loss, _loss = sess.run([train_step, kl_term, rec_loss, loss], 
                                             feed_dict={X: batch})
    if i % save_interval == 0:
        saver.save(sess=sess, save_path=save_path)
        summary = sess.run(merged, feed_dict={X: batch})
        writer.add_summary(summary=summary, global_step=i)
    if i % log_interval == 0:
        random = [np.random.normal(0, 1, latent_dim) for _ in range(12)]
        imgs = sess.run(decoded, feed_dict={encoded: random})
        plot_images(imgs, name='Test images', cmap='Greys', interpolation='bicubic')
    sys.stdout.write('\rIter: {:,}\tKL-Term: {:.4f}\tRec loss: {:.4f}\tLoss:{:.4f}'.format(
        i+1, _kl_loss, _rec_loss, _loss))